IEOR 4500 HW 3 due October 9
Implement the optimal execution algorithm discussed in class, in Python.  
Use the power impact function, where for parameters alpha > 0 and pi > 0, 
when you sell k shares the price (per share) is first reduced by a factor 
of 1 - alpha k^pi
Your algorithm should take as input the values alpha, pi, N (number of 
shares to sell) and T (number of periods available).
Reasonable values for alpha are small, like 1e-3 or 1e-4, and for pi you 
should be able to use any number, but think about 0.5 as a good candidate.
You should eventually be able to run the program using N = 10^4 and T = 
20, but for debugging purposes you should start with much smaller values 
for both N and T.
Try to make your program run as FAST as possible!  Any language is allowed 
so long as it is Python with or without numpy.

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import random
%matplotlib inline

T = 3#Maturity and trading schedule
N = 15#Number of shares to sell
alpha = 0.0001
pi = 0.5

def impact(k):
    return 1-alpha*(k**pi)#This function compute the market impact when we announce we are going to sell k shares.


## Step 2: Optimal execution algorithm
This function is the key component. At time T, we know we must sell all the remaining shares we have to sell. So if we have k shares to sell at time T we will get $k*impact(k)$ in cash. Then going backward in time we compute what is the optimal strategy to sell k shares between $(t,T)$ for all $k \in [0,N]$.

Once we have performed T steps backwards, we look at the optimal path that gives us the best price and how much we can get out of selling those shares. 

In the code, we use memoization to speed up a bit the computation. The revenue is stock in the variable Mem, the different optimal paths (before we can determine which one it is exactly) are stocked in Path.

Inputs:

1. t is the stopping time of the algorithm (remember we are going backward in time)
2. k is the number of remaining shares to sell.
3. Mem is a (T+1)*(N+1) numpy array of -1. Mem[t,k] will correspond to how much cash we can generate by selling k shares between t and T.
4. Path is a (T+1) nupy array of 0 where will store the optimal path.
5. T is the number of trading days (or the number of discreet points at which we can execute trades)
6. N is the number of shares we want to sell

In [7]:
def optimal_execution(t,k,Rev,T,N):
    if (Rev[t,k] != -1):
        return (Rev[t,k],Rev)
    if (t == T):
        Rev[T,k] = impact(k)*k
        return (Rev[T,k],Rev,k)
    else:
        R_max = 0
        i_max = 0
        for i in range(k+1):
            val = impact(i)*i + impact(i)*optimal_execution(t+1,k-i,Rev,T,N)[0]#[0] to get first component of values returned by function
            if(R_max<=val):
                R_max = val
                i_max = i
        Rev[t,k] = R_max
        return (R_max,Rev,i_max)


Step 3: Running the algo to get the optimal execution
The first step is to initialize the numpy array of 0 the variables that will contain the cost and the path at each iteration.
Inputs:
    T is the number of trading days (or the number of discreet points at which we can execute trades)
    N is the number of shares we want to sell
    impact is the price impact fucntion

In [8]:
def initalize(T,N,impact):
    Rev = -1*np.ones((T+1,N+1))
    return Rev
Rev = initalize(T,N,impact)
t0 = time.time()

Ret = optimal_execution(0,N,Rev,T,N)
print("The implicit cost factor is %s .." % str(Ret[0]/N))
print("The number of shares to sell at t=0 is: "+str(Ret[2]))
t1 = time.time()

print("The algo took %s secondes to run..." % str(t1-t0))

The implicit cost factor is 0.9996205954903147 ..
The number of shares to sell at t=0 is: 13
The algo took 0.0 secondes to run...


In [9]:
# #naive approach
# import numpy as np
# test_a = .0001   
# test_N = 10000
# test_pi = .5
# test_T = 20

# def optimal_trade(a,N,pi,T):
#     R = {}    #Dictionary holds the revenue that we get from situation with t time periods left and k assets left
#     V = {}    #Dictionary holds the optimal number of assets we would sell at current time period when we have t time periods left and k assets left
    
#     for k in range(N+1):
#         R[(T,k)] = (1-(a*(k**pi)))*k  #Calculating values for final time period
        
#     for t in range(T,0,-1):  #Looping backwards starting at the end, this is the key to dynamic programming
#         for k in range(N+1):  #Looping through all possible numbers of assets we could have left
#             #print(R[(t,k)])

#             R[(t,k)] = 0      #Start at 0 and continuously improve the optimal value we can get

#             for x in range(k+1):
#                 func = (1-(a*(x**pi)))  #Price impact function
                
#                 if t < T:
#                     value = func*x + func*R[(t+1,k-x)]
#                 else:
#                     value = func*x   #If we are at the last time period, we just sell all the assets 
                
#                 if value > R[(t,k)]:   #If we can improve we we update our two dictionaries
#                     R[(t,k)] = value  
#                     V[(t,k)] = x


#     n = N 
#     t = 1
#     while n > 0:           #This while loop allows us to unwind the dynamic program to find what selling plan we should take 
#         x = V[(t,n)]
#         print('At time:', t, '  Sell:',x)
#         n = n-x
#         t=t+1
#     return R[(1,N)]
        
# R=optimal_trade(test_a,test_N,test_pi,test_T)
# R